In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey
import os
from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load
from scipy.integrate import cumtrapz


In [ ]:
10**(3.75)

In [ ]:
# Gaussian distribution of disc lifetime
mu = 3  # Mean disc lifetime in Myr
sigma = 0.5  # Standard deviation
num_samples = 1000  # Number of Monte Carlo samples
# Generate random tau_disc values from a Gaussian distribution
tau_disc = np.random.normal(mu, sigma, num_samples)

fig,axs = plt.subplots(1, 1, figsize=(8, 6))
plt.hist(tau_disc, bins=50, density=True, alpha=0.6, color='green', edgecolor='black')
axs.axvline(3, color = 'black',  label='Mean tau = 3 Myr')
axs.axvline(3+0.5, color = 'red', linestyle='--')
axs.axvline(3-0.5, color = 'red', linestyle='--')
axs.set_title('Distribution of tau disc Samples', fontsize=18)
plt.savefig('figures/pop_synth/tau_disc_distribution', bbox_inches='tight')

fig,axs = plt.subplots(1, 1, figsize=(8, 6))
import scipy.integrate as scipy
params = code_gas.Params(H_r_model='irradiated')
total_mass_list = []
for tau in tau_disc:
    total_mass, _ = scipy.quad(M_dot_star_linear_scaling, 0.1, tau , args = (params))
    total_mass_list.append(total_mass*u.M_earth.to(u.M_sun))

plt.hist(total_mass_list, bins=50, density=True, alpha=0.6, color='green', edgecolor='black', label = 'Linear scaling')
axs.set_title('Distribution of disc gas Mass Samples, linear scaling', fontsize=18)
axs.set_xlabel('Disc gas mass [M$_\odot$]', fontsize=16)
axs.set_ylabel('Probability Density', fontsize=16)
plt.savefig('figures/pop_synth/mass_disc_distribution', bbox_inches='tight')

import scipy.integrate as scipy
params = code_gas.Params(H_r_model='irradiated', M_dot_gas_star = "Liu_2019")
total_mass_list = []
for tau in tau_disc:
    total_mass, _ = scipy.quad(M_dot_star_t_Mstar, 0.1, tau , args = (params))
    total_mass_list.append(total_mass*u.M_earth.to(u.M_sun))

plt.hist(total_mass_list, bins=50, density=True, alpha=0.6, color='teal', edgecolor='black', label='Liu 2019 scaling')
plt.legend()
axs.set_title('Distribution of disc gas Mass Samples, $M_{\star} = 1 \: M_{\odot}$', fontsize=18)
plt.savefig('figures/pop_synth/mass_disc_distribution_liu', bbox_inches='tight')

fig,axs = plt.subplots(1, 1, figsize=(8, 6))
import scipy.integrate as scipy
params = code_gas.Params(H_r_model='irradiated', star_mass=(0.2*u.M_sun).to(u.M_earth).value)
total_mass_list = []
for tau in tau_disc:
    total_mass, _ = scipy.quad(M_dot_star_linear_scaling, 0.1, tau , args = (params))
    total_mass_list.append(total_mass*u.M_earth.to(u.M_sun))

plt.hist(total_mass_list, bins=50, density=True, alpha=0.6, color='green', edgecolor='black', label = 'Linear scaling')
axs.set_title('Distribution of disc gas Mass Samples, linear scaling', fontsize=18)
axs.set_xlabel('Disc gas mass [M$_\odot$]', fontsize=16)
axs.set_ylabel('Probability Density', fontsize=16)

params = code_gas.Params(H_r_model='irradiated', M_dot_gas_star = "Liu_2019", star_mass=(0.2*u.M_sun).to(u.M_earth).value)
total_mass_list = []
for tau in tau_disc:
    total_mass, _ = scipy.quad(M_dot_star_t_Mstar, 0.1, tau , args = (params))
    total_mass_list.append(total_mass*u.M_earth.to(u.M_sun))

plt.hist(total_mass_list, bins=50, density=True, alpha=0.6, color='teal', edgecolor='black', label='Liu 2019 scaling')
plt.legend()
axs.set_title('Distribution of disc gas Mass Samples, $M_{\star} = 0.2 \: M_{\odot}$', fontsize=18)
plt.savefig('figures/pop_synth/mass_disc_distribution_liu', bbox_inches='tight')

## Irradiated disc, different Mdot star

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['irradiated'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='aquamarine', label='irradiation' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#FF33FF', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#E28D8D', label=str(label[i]))


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)

plt.savefig("figures/pop_synth/planets_metallicity_irradiated", bbox_inches='tight')



## Surface heated disc, different Mdot star

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#00CC88', label='surface heating' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#B89C00', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='brown', label=str(label[i]))


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)

plt.savefig("figures/pop_synth/planets_metallicity_surfheat", bbox_inches='tight')



## Midplane heated disc, different Mdot star

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#006D6F', label='midplane heating' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#B8860B', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#32174D', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#1A1110')


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)

plt.savefig("figures/pop_synth/planets_metallicity_midheat", bbox_inches='tight')



## Cumulative plot with all three different disc models

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/irradiated/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['irradiated'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='aquamarine', label='irradiation' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#FF33FF', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#E28D8D', label=str(label[i]))


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)


folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-','--', '-.']  # Line styles for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#00CC88', label='surface heating' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#B89C00', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='brown', label=str(label[i]))

# Add a title for the entire figure
#fig.suptitle("Planets as a Function of Z and $\dot{M}_{\star}$ for $M_{\star} = 0.5 \: M_{\odot}$", fontsize=18)
# Save the plot

folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/midheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/midheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/midheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/midheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/midheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/midheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/midheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/midheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/midheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/midheat/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/midheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/midheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/midheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/midheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/midheat/Mstar05_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-','--', '-.']  # Line styles for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#006D6F', label='midplane heating' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#B8860B', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#32174D', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#1A1110')

#lower mass M dwarfs!
axs[0].axhline(0.52, color= 'gold', linestyle='--', label='Bonfis et al. 2013')
axs[0].fill_betweenx([1.02, 0.36], 0.01, 0.03, color='gold', alpha=0.1)
axs[0].axhline(0.55, color= 'orange', linestyle='--', label='Sabotta et al. 2015')
axs[0].fill_betweenx([0.96, 0.29], 0.01, 0.03, color='orange', alpha=0.1)

#0.3<M<0.7 M_sun M-dwarfs
axs[0].axhline(0.85, color= 'lightgreen', linestyle='--', label='Pinamonti et al. 2022')
axs[0].fill_betweenx([1.31, 0.64], 0.01, 0.03, color='lightgreen', alpha=0.1)
axs[1].legend()
# Add a title for the entire figure
#fig.suptitle("Planets as a Function of Z and $\dot{M}_{\star}$ for $M_{\star} = 0.5 \: M_{\odot}$", fontsize=18)
# Save the plot


plt.savefig("figures/pop_synth/planets_metallicity_cumulative", bbox_inches='tight')



## Star mass variation


In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear_irr = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]
folder_path_linear_surf = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]
folder_path_liu_irr = [
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]
folder_path_liu_surf = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]

timestep = 5000
Mstar = [0.1, 0.30000000000000004, 0.5, 1.0]   # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])
        print('Loaded', len(simulations[-1]), 'simulations for folder', folder_path)
        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])

    return  planet_dicts

# Load data
H_r_model = ['irradiated'] * 4
planet_dict_liu_irr = load_simulations_and_planet_dict(folder_path_liu_irr, H_r_model, Mstar, timestep)
planet_dict_linear_irr = load_simulations_and_planet_dict(folder_path_linear_irr, H_r_model, Mstar, timestep)
H_r_model = ['Lambrechts_mixed'] * 4
planet_dict_linear_surfheat = load_simulations_and_planet_dict(folder_path_linear_surf, H_r_model, Mstar, timestep)
planet_dict_liu_surfheat = load_simulations_and_planet_dict(folder_path_liu_surf, H_r_model, Mstar, timestep)



In [ ]:
# Plot planet_dict as a function of Mstar
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
dictionaries = [planet_dict_linear_irr,  planet_dict_liu_irr, planet_dict_linear_surfheat, planet_dict_liu_surfheat]
# Plot for t5
ls = ['-', '-.']  # Line styles for different dictionaries
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
#irradiated models
for i, model_dicts in enumerate(dictionaries[:2]):
    axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='aquamarine')
    axs[2].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[3].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#FF33FF', label='CG' if i == 0 else None)
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#E28D8D', label='irradiation' if i == 0 else None)
    axs[0].plot([], [], linestyle = ls[i], color ='#E28D8D', label=str(label[i]))
#surfheat models
for i, model_dicts in enumerate(dictionaries[2:]):
    axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#00CC88')
    axs[2].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#B89C00', label='WJ' if i == 0 else None)
    axs[3].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='brown', label='surface heating' if i == 0 else None)
# Add labels, legend, and grid
axs[3].set_xlabel('$M_{\star}$', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)
    axs[i].set_yscale('log')
    # axs[i].set_xscale('log')
    # axs[i].set_xlim(0.08, 1.2)
plot.Pan25(axs[0])
axs[0].legend(loc='lower right')

## observational constraints
plot.Bonfils13 (axs[0],legend = False) #dubbio
plot.Pinamonti22 (axs[0],legend = False)
plot.Sabotta21 (axs[0], all_stars = True, M_lower_034 = False, M_higher_034=False, SE=True, giants=False,superSE=False, legend = False)
plot.Kunimoto20 (axs[0], F = False, G=True, K=True, SE = True, HJ=False, legend = False)

plot.Sabotta21 (axs[2], all_stars = True, M_lower_034 = True, M_higher_034=True, SE=False, giants=True, superSE=False, legend = False)


axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)
# Set custom tick marks on the x-axis
custom_ticks = [0.1, 0.3, 0.5, 1.0]  # Tick positions
axs[-1].set_xticks(custom_ticks)  # Set the tick positions
axs[-1].set_xticklabels([f'{tick:.1f}' for tick in custom_ticks])  # Set the tick labels
plt.savefig("figures/pop_synth/planets_star_masses_iradiated", bbox_inches='tight')



In [ ]:
# Plot planet_dict as a function of Mstar
fig, axs = plt.subplots(3, 1, figsize=(6, 16), sharex=True)
fig.subplots_adjust(hspace=0)
dictionaries = [planet_dict_linear_irr,  planet_dict_liu_irr, planet_dict_linear_surfheat, planet_dict_liu_surfheat]
# Plot for t5
ls = ['-', '-.']  # Line styles for different dictionaries
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
#irradiated models
for i, model_dicts in enumerate(dictionaries[:2]):
    # axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='aquamarine')
    # axs[1].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[1].plot(Mstar, [(planet_dict['WJ'] + planet_dict['HJ'])/ planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='royalblue', label='giants' if i == 0 else None)
    axs[2].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#FF33FF', label='CG' if i == 0 else None)
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#E28D8D', label='irradiation' if i == 0 else None)
    axs[0].plot([], [], linestyle = ls[i], color ='#E28D8D', label=str(label[i]))
#surfheat models
for i, model_dicts in enumerate(dictionaries[2:]):
    # axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#00CC88')
    # axs[1].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='#B89C00', label='WJ' if i == 0 else None)
    axs[1].plot(Mstar, [(planet_dict['WJ'] + planet_dict['HJ']) / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='navy', label='giants' if i == 0 else None)
    axs[2].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='brown', label='surface heating' if i == 0 else None)
# Add labels, legend, and grid
axs[2].set_xlabel('$M_{\star}$', fontsize=25)
for i in range(3):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)
    axs[i].set_yscale('log')
    # axs[i].set_xscale('log')
    # axs[i].set_xlim(0.08, 1.2)
plot.Pan25(axs[0], SE = True, WHG = False, CG =False,legend = True)
plot.Pan25(axs[1], SE = False, WHG = True, CG =False,legend = True)
plot.Pan25(axs[2], SE = False, WHG = False, CG =True,legend = True)
axs[0].legend(loc='lower right')

## observational constraints
plot.Bonfils13 (axs[0],legend = False) #dubbio
plot.Pinamonti22 (axs[0],legend = False)
plot.Sabotta21 (axs[0], all_stars = True, M_lower_034 = False, M_higher_034=False, SE=True, giants=False,superSE=False, legend = False)
plot.Kunimoto20 (axs[0], F = False, G=True, K=True, SE = True, HJ=False, legend = False)

plot.Sabotta21 (axs[1], all_stars = True, M_lower_034 = True, M_higher_034=True, SE=False, giants=True, superSE=False, legend = False)


axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('hot-warm giants', fontsize=20)
axs[2].set_ylabel('Cold Giants', fontsize=20)
axs[1].set_ylim(6e-4, 5e-1)
axs[2].set_ylim(6e-4, 5e-1)
# Set custom tick marks on the x-axis
custom_ticks = [0.1, 0.3, 0.5, 1.0]  # Tick positions
axs[-1].set_xticks(custom_ticks)  # Set the tick positions
axs[-1].set_xticklabels([f'{tick:.1f}' for tick in custom_ticks])  # Set the tick labels
plt.savefig("figures/pop_synth/planets_star_masses_pan", bbox_inches='tight')



In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
# folder_path_linear_irr = [
#     "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H/5Myrs_fixedMstar_randomZ/Mstar01_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H/5Myrs_fixedMstar_randomZ/Mstar03_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H/5Myrs_fixedMstar_randomZ/Mstar05_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H/5Myrs_fixedMstar_randomZ/Mstar1_random_Z",
# ]
folder_path_linear_surf = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/lowres/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/lowres/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/lowres/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/lowres/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]
# folder_path_liu_irr = [
#     "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
# ]
# folder_path_liu_surf = [
#     "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar01_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar03_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar05_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/3Myrs_fixedMstar_randomZ/Mstar1_random_Z",
# ]

timestep = 5000
Mstar = [0.1, 0.30000000000000004, 0.5, 1.0]   # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])
        print('Loaded', len(simulations[-1]), 'simulations for folder', folder_path)
        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])

    return  planet_dicts

# Load data
H_r_model = ['irradiated'] * 4
#planet_dict_liu_irr = load_simulations_and_planet_dict(folder_path_liu_irr, H_r_model, Mstar, timestep)
planet_dict_linear_irr = load_simulations_and_planet_dict(folder_path_linear_irr, H_r_model, Mstar, timestep)
# H_r_model = ['Lambrechts_mixed'] * 4
# planet_dict_linear_surf = load_simulations_and_planet_dict(folder_path_linear_surf, H_r_model, Mstar, timestep)




In [ ]:
# Plot planet_dict as a function of Mstar
fig, axs = plt.subplots(1, 1, figsize=(5, 5), sharex=True)
fig.subplots_adjust(hspace=0)
dictionaries = [planet_dict_linear_irr]#, planet_dict_linear_surf]
# Plot for t5
ls = ['-', '-.']  # Line styles for different dictionaries
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
c = ['#E28D8D', 'brown']
l = ['irradiation', 'surface heating']
print(dictionaries)
#irradiated models
for i, model_dicts in enumerate(dictionaries):
    axs.plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c[i], label=l[i])
    print(planet_dict['SE'] for planet_dict in model_dicts)
    #axs.plot([], [], linestyle = ls[i], color ='#E28D8D', label=str(label[i]))
    
# Add labels, legend, and grid
axs.set_xlabel('$M_{\star}$', fontsize=25)
axs.tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
axs.tick_params(axis="both", which="minor", direction='in', size=10)
axs.grid(True, linestyle='--', alpha=0.5)
axs.set_yscale('log')
# axs[i].set_xscale('log')
# axs[i].set_xlim(0.08, 1.2)
plot.Pan25(axs, SE=True, WHG=False, CG=False)
axs.legend(loc='lower right')

## observational constraints
plot.Bonfils13 (axs,legend = False) #dubbio
plot.Pinamonti22 (axs,legend = False)
plot.Sabotta21 (axs, all_stars = True, M_lower_034 = False, M_higher_034=False, SE=True, giants=False,superSE=False, legend = False)
plot.Kunimoto20 (axs, F = False, G=True, K=True, SE = True, HJ=False, legend = False)



axs.set_ylabel('super-Earths', fontsize=20)

# Set custom tick marks on the x-axis
custom_ticks = [0.1, 0.3, 0.5, 1.0]  # Tick positions
axs.set_xticks(custom_ticks)  # Set the tick positions
axs.set_xticklabels([f'{tick:.1f}' for tick in custom_ticks])  # Set the tick labels
plt.savefig("figures/pop_synth/planets_star_masses_iradiated_fe", bbox_inches='tight')



In [ ]:
# Plot planet_dict as a function of Mstar
fig, axs = plt.subplots(3, 1, figsize=(6, 16), sharex=True)
fig.subplots_adjust(hspace=0)
dictionaries = [planet_dict_linear_irr,  planet_dict_linear_surf]
# Plot for t5
ls = ['-', '-.']  # Line styles for different dictionaries
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
c_SE = ['#E28D8D', 'brown']
c_giants = ['royalblue', 'navy']
c_CG = ['#FF33FF', 'purple']
l = ['irradiation', 'surface heating']
#irradiated models
for i, model_dicts in enumerate(dictionaries):
    # axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='aquamarine')
    # axs[1].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[1].plot(Mstar, [(planet_dict['WJ'] + planet_dict['HJ'])/ planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_giants[i], label=l[i])
    axs[2].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_CG[i],label=l[i])
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_SE[i],label=l[i])
# Add labels, legend, and grid
axs[2].set_xlabel('$M_{\star}$', fontsize=25)

plot.Pan25(axs[0], SE = True, WHG = False, CG =False,legend = True)
plot.Pan25(axs[1], SE = False, WHG = True, CG =False,legend = True)
plot.Pan25(axs[2], SE = False, WHG = False, CG =True,legend = True)
for i in range(3):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)
    axs[i].set_yscale('log')
    axs[i].legend(loc='lower right')

    # axs[i].set_xscale('log')
    # axs[i].set_xlim(0.08, 1.2)
## observational constraints
plot.Bonfils13 (axs[0],legend = False) #dubbio
plot.Pinamonti22 (axs[0],legend = False)
plot.Sabotta21 (axs[0], all_stars = True, M_lower_034 = False, M_higher_034=False, SE=True, giants=False,superSE=False, legend = False)
plot.Kunimoto20 (axs[0], F = False, G=True, K=True, SE = True, HJ=False, legend = False)

plot.Sabotta21 (axs[1], all_stars = True, M_lower_034 = True, M_higher_034=True, SE=False, giants=True, superSE=False, legend = False)

axs[0].set_title("Linear $\dot{M}_{\star}$ with changing $v_{frag}$", fontsize=18)
axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('hot-warm giants', fontsize=20)
axs[2].set_ylabel('Cold Giants', fontsize=20)
axs[1].set_ylim(6e-4, 5e-1)
axs[2].set_ylim(6e-4, 5e-1)
# Set custom tick marks on the x-axis
custom_ticks = [0.1, 0.3, 0.5, 1.0]  # Tick positions
axs[-1].set_xticks(custom_ticks)  # Set the tick positions
axs[-1].set_xticklabels([f'{tick:.1f}' for tick in custom_ticks])  # Set the tick labels
plt.savefig("figures/pop_synth/planets_star_masses_pan_Fe_H", bbox_inches='tight')



In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for irradiated t3
folder_path_linear_irr = [
    # "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar01_random_Z",
    # "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar03_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar05_random_Z",
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]
# folder_path_linear_surf = [
#     # "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar01_random_Z",
#     # "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar03_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar05_random_Z",
#     "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/alpha_visc/5Myrs_fixedMstar_randomZ/Mstar1_random_Z",
# ]


timestep = 5000
Mstar = [ 0.5, 1.0]   # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and 'N_steps'+str(timestep)+'_Mstar_'+str(mstar) in f and f.endswith('.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])
        print('Loaded', len(simulations[-1]), 'simulations for folder', folder_path)
        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])

    return  planet_dicts

# Load data
H_r_model = ['irradiated'] * 2
#planet_dict_liu_irr = load_simulations_and_planet_dict(folder_path_liu_irr, H_r_model, Mstar, timestep)
planet_dict_linear_irr = load_simulations_and_planet_dict(folder_path_linear_irr, H_r_model, Mstar, timestep)
# H_r_model = ['Lambrechts_mixed'] * 2
# planet_dict_linear_surf = load_simulations_and_planet_dict(folder_path_linear_surf, H_r_model, Mstar, timestep)




In [ ]:
# Plot planet_dict as a function of Mstar
fig, axs = plt.subplots(3, 1, figsize=(6, 16), sharex=True)
fig.subplots_adjust(hspace=0)
dictionaries = [planet_dict_linear_irr]#,  planet_dict_linear_surf]
# Plot for t5
ls = ['-', '-.']  # Line styles for different dictionaries
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
c_SE = ['#E28D8D', 'brown']
c_giants = ['royalblue', 'navy']
c_CG = ['#FF33FF', 'purple']
l = ['irradiation', 'surface heating']
#irradiated models
for i, model_dicts in enumerate(dictionaries):
    # axs[1].plot(Mstar, [planet_dict['HJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='aquamarine')
    # axs[1].plot(Mstar, [planet_dict['WJ'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[1].plot(Mstar, [(planet_dict['WJ'] + planet_dict['HJ'])/ planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_giants[i], label=l[i])
    axs[2].plot(Mstar, [planet_dict['CG'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_CG[i],label=l[i])
    axs[0].plot(Mstar, [planet_dict['SE'] / planet_dict['tot_planets'] for planet_dict in model_dicts], linestyle=ls[i], marker='o', color=c_SE[i],label=l[i])
# Add labels, legend, and grid
axs[2].set_xlabel('$M_{\star}$', fontsize=25)
for i in range(3):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)
    axs[i].set_yscale('log')
    axs[i].legend(loc='lower right')

    # axs[i].set_xscale('log')
    # axs[i].set_xlim(0.08, 1.2)
# ## observational constraints
# plot.Bonfils13 (axs[0],legend = False) #dubbio
# plot.Pinamonti22 (axs[0],legend = False)
# plot.Sabotta21 (axs[0], all_stars = True, M_lower_034 = False, M_higher_034=False, SE=True, giants=False,superSE=False, legend = False)
# plot.Kunimoto20 (axs[0], F = False, G=True, K=True, SE = True, HJ=False, legend = False)

# plot.Sabotta21 (axs[1], all_stars = True, M_lower_034 = True, M_higher_034=True, SE=False, giants=True, superSE=False, legend = False)

axs[0].set_title("Linear $\dot{M}_{\star}$ with higher Fe/H", fontsize=18)
axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('hot-warm giants', fontsize=20)
axs[2].set_ylabel('Cold Giants', fontsize=20)
axs[1].set_ylim(6e-4, 5e-1)
axs[2].set_ylim(6e-4, 5e-1)
# Set custom tick marks on the x-axis
custom_ticks = [0.1, 0.3, 0.5, 1.0]  # Tick positions
axs[-1].set_xticks(custom_ticks)  # Set the tick positions
axs[-1].set_xticklabels([f'{tick:.1f}' for tick in custom_ticks])  # Set the tick labels
plt.savefig("figures/pop_synth/planets_star_masses_fe7", bbox_inches='tight')



In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Fe_H_07/5Myrs_fixedMstar_randomZ/Mstar1_random_Z",
]


timestep = 5000
H_r_model = ['irradiated']*250
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))

for j in range(len(simulations[0])):
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
norm = mpl.colors.LogNorm(vmin=sim_params.t_in, vmax=sim_params.t_fin)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('Accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, liu,  $M_{\star}=1 \: M_{\odot}$ , random Z, random $M_{\mathrm{disc}}$, 1 planet", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random_alpha_1", bbox_inches='tight')